In [1]:
from openmdao.visualization.set_values_gui.set_values_gui import SetValuesUI

In [2]:
import openmdao.api as om

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import importlib
import openmdao.visualization.set_values_gui.set_values_gui
importlib.reload(openmdao.visualization.set_values_gui.set_values_gui)

<module 'openmdao.visualization.set_values_gui.set_values_gui' from '/Users/hschilli/Documents/OpenMDAO/dev/I2239-value-set-with-widgets/openmdao/visualization/set_values_gui/set_values_gui.py'>

In [5]:
from openmdao.test_suite.components.sellar_feature import SellarMDA
prob = om.Problem()
prob.model = SellarMDA()

prob.model.set_input_defaults('x', val=1., units='m')


prob.driver = om.ScipyOptimizeDriver(optimizer='SLSQP', tol=1e-8)

prob.model.add_design_var('x', lower=0, upper=10)
prob.model.add_design_var('z', lower=0, upper=10)
prob.model.add_objective('obj')
prob.model.add_constraint('con1', upper=0)
prob.model.add_constraint('con2', upper=0)

# Ask OpenMDAO to finite-difference across the model to compute the gradients for the optimizer
prob.model.approx_totals()

# prob['y2'] = 3.4

In [6]:
prob.setup()

/Users/hschilli/Documents/OpenMDAO/dev/I2239-value-set-with-widgets/openmdao/core/group.py:1933: UnitsWarning:<model> <class SellarMDA>: Output '_auto_ivc.v1' with units of 'm' is connected to input 'cycle.d1.x' which has no units.
/Users/hschilli/Documents/OpenMDAO/dev/I2239-value-set-with-widgets/openmdao/core/group.py:1933: UnitsWarning:<model> <class SellarMDA>: Output '_auto_ivc.v1' with units of 'm' is connected to input 'obj_cmp.x' which has no units.


In [7]:
ui = SetValuesUI(prob, vars_to_set=['x','z', 'y2'])
ui.setup()

setup with autoreload 2 Output(layout=Layout(border='1px solid black', width='60%'))


In [8]:
prob.model.get_io_metadata(('input',), ['units', ],
                                     get_remote=True,
                                     return_rel_names=False)

{'cycle.d1.z': {'units': None, 'prom_name': 'z', 'discrete': True},
 'cycle.d2.z': {'units': None, 'prom_name': 'z', 'discrete': True},
 'obj_cmp.z': {'units': None, 'prom_name': 'z', 'discrete': True},
 'cycle.d1.x': {'units': None, 'prom_name': 'x', 'discrete': True},
 'obj_cmp.x': {'units': None, 'prom_name': 'x', 'discrete': True},
 'cycle.d1.y2': {'units': None, 'prom_name': 'y2', 'discrete': True},
 'obj_cmp.y2': {'units': None, 'prom_name': 'y2', 'discrete': True},
 'con_cmp2.y2': {'units': None, 'prom_name': 'y2', 'discrete': True},
 'cycle.d2.y1': {'units': None, 'prom_name': 'y1', 'discrete': True},
 'obj_cmp.y1': {'units': None, 'prom_name': 'y1', 'discrete': True},
 'con_cmp1.y1': {'units': None, 'prom_name': 'y1', 'discrete': True}}

In [9]:
ui.display()

In [10]:
prob['y1']

array([1.])

In [11]:
# prob.run_driver()

In [12]:
class SpeedComp(om.ExplicitComponent):
    """Simple speed computation from distance and time with unit conversations."""

    def setup(self):
        self.add_input('distance', val=1.0, units='km')
        self.add_input('time', val=1.0, units='h')
        self.add_output('speed', val=1.0, units='km/h')

    def compute(self, inputs, outputs):
        outputs['speed'] = inputs['distance'] / inputs['time']
prob = om.Problem()
prob.model.add_subsystem('c1', SpeedComp(), promotes=['*'])
prob.model.add_subsystem('c2', om.ExecComp('f=speed',speed={'units': 'm/s'}), promotes=['*'])

prob.model.set_input_defaults('distance', val=1., units='m')
prob.model.set_input_defaults('time', val=1., units='s')

# prob.model.connect('c1.speed', 'speed')

prob.setup()


In [13]:
ui = SetValuesUI(prob, vars_to_set=['time', 'speed'])
ui.setup()

setup with autoreload 2 Output(layout=Layout(border='1px solid black', width='60%'))


In [14]:
ui.display()

In [15]:
prob.model.list_inputs()

3 Input(s) in 'model'

varname     val 
----------  ----
c1
  distance  [1.]
  time      [1.]
c2
  speed     [1.]




[('c1.distance', {'val': array([1.])}),
 ('c1.time', {'val': array([1.])}),
 ('c2.speed', {'val': array([1.])})]

In [16]:
prob['c1.distance']

array([1.])